# Application of amplitude estimation to Finances: Standard Approach Problems

In notebook *08_ApplicationTo_Finance_01_StandardApproach* we explain how to use **amplitude amplifications** techniques to compute expectation values and in the notebook: *09_ApplicationTo_Finance_02_Call_Option_BlackScholes* we use this for developing a complete price estimation for an *european call option* under the **Black Scholes** model.

In this notebook we are going to point out several problems of this standard approach and give some proposal solutions in order to avoid them.


In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm
from copy import deepcopy

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Amplitude Amplification and Expectation value computations.

As explained in notebook: *08_ApplicationTo_Finance_01_StandardApproach* the idea is compute the expectation of a function  $f(x)$ when $x$ follows proability density $p(x)$:

$$\mathbb{E}[f]=\int_a^bp(x)f(x)dx$$

that can be expresed as a Riemman sum:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)$$

using **amplitude amplification** algorithms. 

In this kind of algorithms we have a quantum state $|\Psi\rangle$ that can be decomposed in the following way:

$$|\Psi\rangle=\sqrt{a}|\Psi_{0}\rangle+\sqrt{1-a}|\Psi_{1}\rangle$$

where:
$$
\begin{array}{l}
&\sqrt{a}|\Psi_{0}\rangle = |0\rangle \otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n},\\\\
&\sqrt{1-a}|\Psi_{1}\rangle = |1\rangle\otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)(1-f(x_i))}|i\rangle_{n}.
\end{array}
$$
The probability of measuring $|0\rangle$ in the leftmost qubit is:

$$ a = \sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2,$$

## 2. Problems of the standard procedure

One of the major problems of the procedure summarize in Section 1 is the following:

If we have negative values in $f$, we won't be able to compute the correct sum because, when we measure the leftmost qubit we are taking the absolute value:

$$\sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2 = \sum_{i=0}^{2^{n}-1}p(x_i)\left|f(x_i)\right| $$

For demonstrating this we are going to develop following example:

* Domain: our $x$ will be a set of $2^{n}$ integers numbers.
* $p(x)$: Over our domain we are going to define a properly normalised density distribution in the form:
$$p(x)=\frac{x}{\sum_{i=0}^{2^{n}-1}i}$$

we are going to use $n=3$ and the following function $f(x)$:

$$f = \dfrac{1}{7}\left(-0,-1,-2,-3,4,5,6,7\right).$$


In [ ]:
n = 3
N = 2**n
x = np.arange(N)

p_X = x/np.sum(x)

f_X = np.copy(p_X)
f_X[1] = -p_X[1]
f_X[2] = -p_X[2]
f_X[3] = -p_X[3]
print(f_X)

Now we load the data using the usual way

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array, uniform_distribution

In [ ]:
p_gate = load_probability(p_X)
#For avoiding problems we need to provide absolute values of f
f_gate = load_array(np.sqrt(np.abs(f_X)))


oracle_problem = qlm.QRoutine()
register_problem = oracle_problem.new_wires(n+1)
oracle_problem.apply(p_gate, register_problem[:n])
oracle_problem.apply(f_gate, register_problem)

%qatdisplay oracle_problem --svg

In [ ]:
#Testing normalisation conditions
print('p(x) condition: {}'.format(np.isclose(np.sum(p_X), 1)))
print('f(x) condition: {}'.format(np.max(f_X) <= 1))

In [ ]:
print("Exact solution: ",np.dot(p_X,f_X))

Now we can use the **amplitude estimation** algorithms for calculating the desired integral

In [ ]:
target = [0]
index = [oracle_problem.arity-1]

In [ ]:
from QQuantLib.AE.maximum_likelihood_ae import MLAE

mlae_dict = {
    'qpu': linalg_qpu,
}
mlae = MLAE(
    oracle_problem,
    target = target,
    index = index, 
    **mlae_dict
)

mlae_a = mlae.run()
print('mlae_a: ', mlae_a)

In [ ]:
from QQuantLib.AE.ae_classical_qpe import CQPEAE

ae_cqpe_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 8,
    'shots': 100
}

ae_cqpe = CQPEAE(
    oracle_problem,
    target = target,
    index = index, 
    **ae_cqpe_dict
)
ae_cqpe_a  = ae_cqpe.run()

print('ae_cqpe_a: ', ae_cqpe_a)

In [ ]:
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 8,
    'shots': 10
}

ae_iqpe = IQPEAE(
    oracle_problem,
    target = target,
    index = index, 
    **ae_iqpe_dict
)

ae_iqpe_a  = ae_iqpe.run()

print('ae_iqpe_a: ', ae_iqpe_a)

In [ ]:
from QQuantLib.AE.iterative_quantum_ae import IQAE

iqae_dict = {
    'qpu': linalg_qpu
}

iqae = IQAE(
    oracle_problem,
    target = target,
    index = index, 
    **iqae_dict
)

iqae_a = iqae.run()

print('iqae_a: ', iqae_a)

In [ ]:
methods = ['MLAE', 'CQPEAE', 'IQPEAE', 'IQAE']
a_estimated = [mlae.ae, ae_cqpe.ae, ae_iqpe.ae, iqae.ae]


dic_staff = {
    'AE_a': a_estimated,
}

Results = pd.DataFrame(dic_staff, index=methods)
Results['Exact_Solution'] = np.dot(p_X,f_X)

In [ ]:
Results

**As can be seen none of all the used methods give us the correct answer!**

## 3. New Loading Data Procedure

The reason for the fail showed in Section 2 is our loading data procedure. If we review the notebook: *01_Data_Loading_Module_Use* we see that our *load_probability*  function from the **DL/data_loading** module loads the $\sqrt{p(x)}$. Additionally when we load the function $f(x)$ we, really, load the $\sqrt{f(x)}$. 

On easy workarond for dealing with section 2 problem is changing our data loading procedure in the following ways:
* We are going to load $p(x)$ and $f(x)$ instead of $\sqrt{p(x)}$ and $\sqrt{f(x)}$
* The probability distribution will be loaded as a function insted of using the probability density procedure.

If we have discretized the $p(x)$ and the $f(x)$ functions in $2^{n}$ values this new loading procedure will need $n+2$ qbits instead of the $n+1$ qbit of the original procedure.

Here we resumen the new loading data protocol:


1. We begin with a $n+2$ qbits state (the 1 and 2 superscript is for indetifiyng the qbit, and the $n$ subscript is for spcifing a n qbits state)

$$|0\rangle^1 \otimes |0\rangle^2 \otimes|0\rangle_{n}$$

2. We apply uniform distribution over the $n$ qbits state (the 1 and 2 subscripts are for indicating over each qbit the operator should be applied)

$$\big(I_1 \otimes I_2 \otimes H^{\otimes n}\big)\big(|0\rangle^1 \otimes |0\rangle^2 \otimes|0\rangle_{n}\big) = |0\rangle^1 \otimes |0\rangle^2 \otimes H^{\otimes n}|0\rangle_{n}=$$
$$=|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|i\rangle_{n}$$



3. We apply the **loading function operator** for loading the probability distribution $p(X)$: $\mathcal{F(p)}$ over the qbit $|0\rangle^2$

$$\big(I_1 \otimes \mathcal{F_2(p)}\big) \big(|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|i\rangle_{n} \big) =$$
$$= |0\rangle^1 \otimes \mathcal{F_2(p)} \big( |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|i\rangle_{n} \big)= $$

$$=|0\rangle^1 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|i\rangle_{n} \otimes [ p(i) |0\rangle^2 +  \sin(\theta_{p(i)}) |1\rangle^2] =$$


$$=|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n} +  |0\rangle^1 \otimes |1\rangle^2 \otimes ...$$
4. In the last expresion we only are interested in the terms with $|0\rangle^1 \otimes |0\rangle^2$ the other states do not interest us, so in the following steps we delete them from the formulas. So the importan part will be:

$$|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n}$$

5. Now we apply the **loading function operator** for loading $f(x)$: $\mathcal{F(f)}$ over the qbit $|0\rangle^1$

$$\big(I_2 \otimes \mathcal{F_1(f)}\big) \big(|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n} \big)=$$

$$=|0\rangle^2 \otimes \mathcal{F_1(f)} \big(|0\rangle^1 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n}\big)=$$


$$=|0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n} \otimes [ f(i) |0\rangle^1 +  \sin(\theta_{f(i)}) |1\rangle^1] = $$

$$=|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)|i\rangle_{n} + |1\rangle^1 \otimes |0\rangle^2 \otimes ...$$

6. Again we are interested only on the therms with $|0\rangle^1 \otimes |0\rangle^2$ so we take off from the formulas the other terms. So the important part will be:

$$|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)|i\rangle_{n}$$

7. Finally we apply another uniform distribution over the $n$ qbits state:

$$\big(I_1 \otimes I_2 \otimes H^{\otimes n}\big) \big(|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)|i\rangle_{n}\big) =$$
$$=|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)H^{\otimes n}|i\rangle_{n}$$


The final state of our new loading protocol will be in the form:

$$|\Psi \rangle = |0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)H^{\otimes n}|i\rangle_{n} + |0\rangle^1 \otimes |1\rangle^2 \otimes ... + |1\rangle^1 \otimes |0\rangle^2 \otimes ... + |1\rangle^1 \otimes |1\rangle^2 \otimes ...$$

The uniform distribution acting over any $|i\rangle_n$ state can be expresed as:

$$H^{\otimes n}|i\rangle_{n} = \frac{1}{\sqrt{2^n}}\sum_{j=0}^{2^{n}-1} (-1)^{ij}|j\rangle_n=\frac{1}{\sqrt{2^n}}[|0\rangle_n + \sum_{j=1}^{2^{n}-1} (-1)^{ij}|j\rangle_n]$$ 

So replacing it in our final state:

$$|\Psi \rangle = |0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{2^n} \sum_{i=0}^{2^{n}-1}p(i)f(i)|0\rangle_n + |0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{2^n}\sum_{j=1}^{2^{n}-1} (-1)^{ij}p(i)f(i)|j\rangle_n + |0\rangle^1 \otimes |1\rangle^2 \otimes ... + |1\rangle^1 \otimes |0\rangle^2 \otimes ... + |1\rangle^1 \otimes |1\rangle^2 \otimes ...$$


As can be seen the term that carries the information we need is: $|0\rangle^1 \otimes |0\rangle^2 \otimes|0\rangle_n$. We do not care from other terms so we can write the final state in the following way:


$$|\Psi \rangle = \frac{1}{2^n}\sum_{i=0}^{2^{n}-1}p(i)f(i)|0\rangle^1 \otimes |0\rangle^2 \otimes|0\rangle_n+...$$

All this procedure is implemented in the following cell:

In [ ]:
new_oracle = qlm.QRoutine()
#For new data loading procedure we need n+2 qbits
new_registers = new_oracle.new_wires(n+2)
#Step 2 of Procedure: apply Uniform distribution 
new_oracle.apply(uniform_distribution(n),new_registers[:n])
#Step 3 of Procedure: apply loading function operator for loading p(x)
new_p_gate = load_array(p_X, id_name = 'p(x)')
new_oracle.apply(new_p_gate, [new_registers[:n], new_registers[n]])
#Step 5 of Procedure: apply loading function operator for loading f(x)
new_f_gate = load_array(f_X, id_name = 'f(x)')
new_oracle.apply(new_f_gate, [new_registers[:n], new_registers[n+1]])
#Step 7 of Procedure: apply Uniform distribution 
new_oracle.apply(uniform_distribution(n),new_registers[:n])

In [ ]:
%qatdisplay new_oracle --svg

Now with the new loading procedure we can again express the quantum state as a linear combination of two orthogonal states $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$:

$$|\Psi\rangle = \sqrt{a}|\Psi_{0}\rangle+\sqrt{1-a}|\Psi_{1}\rangle$$

where:
$$\sqrt{a}|\Psi_{0}\rangle = \left[\dfrac{1}{2^n}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right]|0\rangle^1 \otimes|0\rangle^2 \otimes|0\rangle_n$$

Now we can remove the super-index from 1 qbit $|0\rangle$ state:

$$\sqrt{a}|\Psi_{0}\rangle = \left[\dfrac{1}{2^n}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right]|0\rangle \otimes|0\rangle \otimes|0\rangle_n$$
 

The probability of measuring $|0\rangle \otimes|0\rangle \otimes|0\rangle_n$ is:

$$ a= \left|\dfrac{1}{N}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right|^2,$$

so then computing $\sqrt{a}$ gives us an estimation.

Now we can use again the *amplitude estimation* routines, but using the right target and index!!

In [ ]:
new_target = [0 for i in range(new_oracle.arity)]
print('new_target: ', new_target)
new_index = [i for i in range(new_oracle.arity)]
print('new_index: ', new_index)

In [ ]:
from QQuantLib.AE.maximum_likelihood_ae import MLAE

m_k = [0, 1, 10, 20, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50]
n_k = [100 for i in m_k]


mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': [m_k, n_k]
}
mlae = MLAE(
    new_oracle,
    target = new_target,
    index = new_index, 
    **mlae_dict
)

mlae_a = mlae.run()
print('mlae_a: ', mlae_a)

In [ ]:
from QQuantLib.AE.ae_classical_qpe import CQPEAE

ae_cqpe_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 12,
    'shots': 50
}

ae_cqpe = CQPEAE(
    new_oracle,
    target = new_target,
    index = new_index, 
    **ae_cqpe_dict
)
ae_cqpe_a  = ae_cqpe.run()

print('ae_cqpe_a: ', ae_cqpe_a)

In [ ]:
ae_cqpe.run_time

In [ ]:
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 12,
    'shots': 10
}

ae_iqpe = IQPEAE(
    new_oracle,
    target = new_target,
    index = new_index, 
    **ae_iqpe_dict
)

ae_iqpe_a  = ae_iqpe.run()

print('ae_iqpe_a: ', ae_iqpe_a)

In [ ]:
ae_iqpe.run_time

In [ ]:
from QQuantLib.AE.iterative_quantum_ae import IQAE

epsilon = 0.001
iqae_dict = {
    'qpu': linalg_qpu,
    'epsilon': epsilon
}

iqae = IQAE(
    new_oracle,
    target = new_target,
    index = new_index,  
    **iqae_dict
)

iqae_a = iqae.run()

print('iqae_a: ', iqae_a)

**BE AWARE**

Now the **amplitude estimation** routines will estimate:

$$ a= \left|\dfrac{1}{2^n}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right|^2$$

so if we measured $a$ for getting the proper result we need:

$$\sum_{i=0}^{2^{n}-1}p(i)f(i) = 2^n * \sqrt{a}$$

So in order to get the correct a we must provide the $2^n\sqrt{a}$

In [ ]:
methods = ['MLAE', 'CQPEAE', 'IQPEAE', 'IQAE']
a_estimated = [mlae.ae, ae_cqpe.ae, ae_iqpe.ae, iqae.ae]


dic_staff = {
    'AE_a': np.sqrt(a_estimated)*2**n,
}

new_Results = pd.DataFrame(dic_staff, index=methods)
new_Results['Exact_Solution'] = np.dot(p_X,f_X)

In [ ]:
new_Results

## 4. More Problems

With our new data loading procedure we have solve the issue from Section 2 but we still face one major problem: when the expectation of $f$ is negative, we won't be able to notice it, again because we are taking the absolute value. 

To demonstrate it let us define a new linear function $f$ which is negative in the second half:
$$f = \dfrac{1}{7}\left(0,1,2,3,-4,-5,-6,-7\right).$$

In [ ]:
new_fx = np.copy(f_X)
new_fx[4] = -f_X[4]
new_fx[5] = -f_X[5]
new_fx[6] = -f_X[6]
new_fx[7] = -f_X[7]

In [ ]:
#Testing normalisation condition
print('f(x) condition: {}'.format(np.max(new_fx) <= 1))
print('f(x) condition: {}'.format(np.min(new_fx) >= -1))

In [ ]:
problem_oracle = qlm.QRoutine()
#For new data loading procedure we need n+2 qbits
problem_registers = problem_oracle.new_wires(n+2)
#Step 2 of Procedure: apply Uniform distribution 
problem_oracle.apply(uniform_distribution(n), problem_registers[:n])
#Step 3 of Procedure: apply loading function operator for loading p(x)
new_p_gate = load_array(p_X, id_name = 'p(x)')
problem_oracle.apply(new_p_gate, [problem_registers[:n], problem_registers[n]])
#Step 5 of Procedure: apply loading function operator for loading f(x)
problem_f_gate = load_array(new_fx, id_name = 'f(x)')
problem_oracle.apply(problem_f_gate, [problem_registers[:n], problem_registers[n+1]])
#Step 7 of Procedure: apply Uniform distribution 
problem_oracle.apply(uniform_distribution(n),problem_registers[:n])

%qatdisplay problem_oracle --svg


problem_target = [0 for i in range(problem_oracle.arity)]
print('problem_target: ', problem_target)
problem_index = [i for i in range(problem_oracle.arity)]
print('problem_index: ', problem_index)

In [ ]:
from QQuantLib.AE.maximum_likelihood_ae import MLAE

m_k = [0, 1, 10, 20, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50]
n_k = [100 for i in m_k]


mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': [m_k, n_k]
}
mlae = MLAE(
    problem_oracle,
    target = problem_target,
    index = problem_index, 
    **mlae_dict
)

mlae_a = mlae.run()
print('mlae_a: ', mlae_a)

In [ ]:
from QQuantLib.AE.ae_classical_qpe import CQPEAE

ae_cqpe_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 12,
    'shots': 10
}

ae_cqpe = CQPEAE(
    problem_oracle,
    target = problem_target,
    index = problem_index, 
    **ae_cqpe_dict
)
ae_cqpe_a  = ae_cqpe.run()

print('ae_cqpe_a: ', ae_cqpe_a)

In [ ]:
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 12,
    'shots': 10
}

ae_iqpe = IQPEAE(
    problem_oracle,
    target = problem_target,
    index = problem_index, 
    **ae_iqpe_dict
)

ae_iqpe_a  = ae_iqpe.run()

print('ae_iqpe_a: ', ae_iqpe_a)

In [ ]:
from QQuantLib.AE.iterative_quantum_ae import IQAE

epsilon = 0.001
iqae_dict = {
    'qpu': linalg_qpu,
    'epsilon': epsilon
}

iqae = IQAE(
    problem_oracle,
    target = problem_target,
    index = problem_index,  
    **iqae_dict
)

iqae_a = iqae.run()

print('iqae_a: ', iqae_a)

In [ ]:
methods = ['MLAE', 'CQPEAE', 'IQPEAE', 'IQAE']
a_estimated = [mlae.ae, ae_cqpe.ae, ae_iqpe.ae, iqae.ae]


dic_staff = {
    'AE_a': np.sqrt(a_estimated)*2**n,
}

problem_Results = pd.DataFrame(dic_staff, index=methods)
problem_Results['Exact_Solution'] = np.dot(p_X,new_fx)

In [ ]:
problem_Results

## 5. RQAE solution

In order to solve the problem presented in Section we need an algorithm which distinguishes the sign of the underlying amplitude. For that we propose the RQAE, for more information check the corresponding notebook and article.

In [ ]:
from QQuantLib.AE.real_quantum_ae import RQAE

In [ ]:
q = 2
epsilon = 0.001
gamma = 0.05 
rqae_dict = {
    'qpu': linalg_qpu,
    'epsilon': epsilon,
    'gamma': gamma,
    'q': q
}

rqae = RQAE(
    problem_oracle,
    target = problem_target,
    index = problem_index,
    **rqae_dict
)

In [ ]:
rqae_a = rqae.run()
print('rqae_a: ', rqae_a)

**BE AWARE**

The **RQAE** algorithm provide us, directly, the amplitude of the $|0\rangle \otimes|0\rangle \otimes|0\rangle_n$ state (the other algorithms provide us the probability so we need to do the square root).

So the **RQAE** algorithm give as an estimation of $a$ (as an amplitude) and the correspondent expectation will be:

$$\sum_{i=0}^{2^{n}-1}p(i)f(i) = 2^n * a$$

In [ ]:
methods = ['MLAE', 'CQPEAE', 'IQPEAE', 'IQAE', 'RQAE']
a_estimated = [
    2**n*np.sqrt(mlae.ae), 
    2**n*np.sqrt(ae_cqpe.ae), 
    2**n*np.sqrt(ae_iqpe.ae), 
    2**n*np.sqrt(iqae.ae), 
    2**n*rqae.ae #Here amplitude is provided by RQAE
]


dic_staff = {
    'AE_a': a_estimated,
}

problem_Results = pd.DataFrame(dic_staff, index=methods)
problem_Results['Exact_Solution'] = np.dot(p_X,new_fx)

In [ ]:
problem_Results